## Photometry on MIRI images

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS, FITSFixedWarning
from astropy.nddata import Cutout2D
from astropy.table import Table
import astropy.units as u
from regions import EllipsePixelRegion, PixCoord
from photutils.aperture import aperture_photometry
from photutils.aperture import EllipticalAperture
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import warnings
import subprocess
import json

warnings.simplefilter("ignore", category=FITSFixedWarning)


Let's start by calculating the angles of all master FITS files:

In [ ]:
def calculate_angle(fits_file):
    """A function that reads in the header of a .fits file and extracts the information
        about the rotation of the image with respect to the N and E directions.

    Args:
        fits_file (string): The .fits file to be rotated in the next steps
    """
    with fits.open(fits_file) as hdul:
        header = hdul[1].header

        # Extract rotation angle from PC matrix
        if 'PC1_1' in header and 'PC2_2' in header:
            cost = header['PC1_1'] 
            sint = header['PC2_1'] 

            # 180 - angle takes care of X-axis flipping in sky coordinates!
            angle = 180 - np.arccos(cost) * 180 / np.pi
            
            # Restore quadrant information since cos is symmetric
            if sint < 0:
                angle = -angle
            
            #print(f"The image {fits_file} is rotated by {angle:.2f} degrees with respect to North")        
        else:
            print("No PC matrix found, assuming no rotation")
            angle = 0
        
    return angle

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
primer003_f770w = glob.glob(os.path.join(cutout_dir, '*F770W*003.fits'))
primer003_f1800w = glob.glob(os.path.join(cutout_dir, '*F1800W*003.fits'))
primer004_f770w = glob.glob(os.path.join(cutout_dir, '*F770W*004.fits'))
primer004_f1800w = glob.glob(os.path.join(cutout_dir, '*F1800W*004.fits'))
cweb1 = glob.glob(os.path.join(cutout_dir, '*cweb1.fits'))
cweb2 = glob.glob(os.path.join(cutout_dir, '*cweb2.fits'))

data = [primer003_f770w, primer003_f1800w, primer004_f770w, primer004_f1800w, cweb1, cweb2]

for survey in data:
    angles = []
    for cutout in survey:
        angle = calculate_angle(cutout)
        angles.append(angle)
    mean_angle = np.mean(angles)
    std_angle = np.std(angles, ddof=1)
    print(os.path.basename(str(survey)))
    print('The mean angle is: ', mean_angle, ' +/- ', std_angle)

Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder, save_plot=False):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    galaxy_id = int(galaxy_id)
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"]
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])

    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.031    # arcsec/pixel
    miri_scale = 0.10996    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta = -row['Apr_Theta']
    theta_new = ((theta - angle) % 180) * u.deg
    
    # --- Create region file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    reg_file = os.path.join(output_folder, f'regions/{galaxy_id}_{survey}{obs}_aperture.reg') 
    
    # --- Write to DS9-compatible region file ---
    with open(reg_file, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_new:.2f})\n")

    if save_plot:
        
        # --- Clean and prepare the MIRI data for plotting ---
        miri_clean = np.copy(miri_data)
        finite_vals = miri_clean[np.isfinite(miri_clean)].flatten()
        
        # Sort and get lowest 80% values
        sorted_vals = np.sort(finite_vals)
        cutoff_index = int(0.8 * len(sorted_vals))
        background_vals = sorted_vals[:cutoff_index]
        background_mean = np.mean(background_vals)

        # Replace NaNs or infs with background mean
        miri_clean[~np.isfinite(miri_clean)] = background_mean

        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(miri_clean, origin='lower', cmap='gray', vmin=np.percentile(miri_clean, 5), vmax=np.percentile(miri_clean, 99))

        # Original ellipse (without rotation correction)
        ellipse_original = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta,  # Just the original θ!
            edgecolor='red',
            facecolor='none',
            lw=2,
            label='Original Ellipse'
        )
        ax.add_patch(ellipse_original)

        ellipse = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta_new.to_value(u.deg),
            edgecolor='blue',
            linestyle='--',  # maybe dashed to differentiate
            facecolor='none',
            lw=2,
            label='Rotated Ellipse'
        )
        ax.add_patch(ellipse)
        
        
        
        ax.set_title(f"Galaxy {galaxy_id} - {filter} ({survey}{obs})")
        ax.set_xlim(miri_x - 30, miri_x + 30)
        ax.set_ylim(miri_y - 30, miri_y + 30)
        ax.legend(loc='upper right')
        
        # Save figure
        png_path = os.path.join(output_folder, f'masks/{galaxy_id}_{survey}{obs}_aperture_overlay.png')
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
    

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call function
    adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=True)


Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"

id = '19393'
filter = 'F770W'
survey_obs = 'primer003'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
#subprocess.run(["ds9", cutout_path, "-regions", reg_path])

id = '9871'
filter = 'F1800W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
#subprocess.run(["ds9", cutout_path, "-regions", reg_path])

id = '8469'
filter = 'F770W'
survey_obs = 'cweb1'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
#subprocess.run(["ds9", cutout_path, "-regions", reg_path])

id = '20397'
filter = 'F770W'
survey_obs = 'cweb2'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
csv_file = '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'

# Select only the Apr_* columns and ID
selected_cols = table["Apr_A", "Apr_B", "Apr_Xcenter", "Apr_Ycenter", "Apr_Theta", "ID"]

selected_cols.write(csv_file, format="csv", overwrite=True)